In [1]:
import os
import pathlib
import soundfile as sf
import pandas as pd
from tqdm import tqdm

In [2]:
lstm_path = '/home/yangwenhao/project/lstm_speaker_verification'
root_path = '/home/yangwenhao/project/SpeakerVerification-pytorch'

In [2]:
miss_uids = set([])
with open('/home/work2020/yangwenhao/project/lstm_speaker_verification/data/vox2/dev/miss_uids', 'r') as f:
    for l in f.readlines():
        uid = l.rstrip('\n')
        miss_uids.add(uid)
        
with open('/home/work2020/yangwenhao/project/lstm_speaker_verification/data/vox2/dev/wav.scp', 'r') as f:
    for l in tqdm(f.readlines()):
        uid, upath = l.split()
        if uid in miss_uids:
            dest_path = upath.replace('/home/storage/yangwenhao/dataset/voxceleb2', '/home/ssd2020/yangwenhao/dataset/voxceleb2_part2')
            if not os.path.exists(os.path.dirname(dest_path)):
                os.makedirs(os.path.dirname(dest_path))
                
            os.system('cp %s %s'%(upath, dest_path))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1092009/1092009 [53:02<00:00, 343.14it/s]


In [ ]:
# split -b 19G -d voxceleb2_part2.tar.gz voxceleb2_part2.tar.gz
# bypy download datasets/voxceleb/voxceleb2.tar.bz04 ./

In [4]:
dataset = 'cnceleb'
subset = 'dev'

In [5]:
utt2dur = {}
with open(lstm_path + '/data/{}/dev/utt2dur'.format(dataset), 'r') as f:
    for l in tqdm(f.readlines(), ncols=50):
        uid, dur = l.split()
        utt2dur[uid] = float(dur)

utt2dur_keys = set(utt2dur.keys())
utt2path = {}
with open(lstm_path + '/data/{}/dev/wav.scp'.format(dataset), 'r') as f:
    for l in tqdm(f.readlines(), ncols=50):
        lst = l.split()
        if len(lst) == 2:
            uid, upath = lst
        elif len(lst) == 3:
            uid, _, upath = lst
        elif len(lst) > 3:
            uid   = lst[0]
            upath = lst[4]
        
        utt2path[uid] = upath
        if uid not in utt2dur_keys:
            print(uid)
            data,sr = sf.read(upath)
            dur = len(data)/sr
            utt2dur[uid] = dur

100%|█| 107953/107953 [00:00<00:00, 1106167.88it/s
100%|█| 107953/107953 [00:00<00:00, 658190.54it/s]


In [25]:
errors = []
with open(root_path + '/Data/checkpoint/ThinResNet18/Mean_batch256_k7_seblock_downk1_avg1_SAP2_em256_dp05_alpha0_none1_wde5_2sesmix8/arcsoft_adam_cyclic/cnceleb/wave_fb80_dist2/train.csv', 'r') as f:
    lst = list(f.readlines())
    # print(lst[:1])
    for l in tqdm(lst[1:]):
        uid, start, end = l.split(',')
        if uid != 'uid':
            start = int(start)
            end   = int(end.rstrip('\n'))
            upath = utt2path[uid]
            try:
                data,sr = sf.read(upath, start=start, stop=end)
            except Exception as e:
                errors.append(l)

100%|██████████| 612096/612096 [51:45<00:00, 197.12it/s]  


In [26]:
errors

['id00635-live_broadcast-01-221,0,208000\n']

In [7]:
utt2path['id00635-live_broadcast-01-221']

'/home/yangwenhao/dataset/CN-Celeb/data/id00635/live_broadcast-01-221.flac'

In [13]:
data,sr = sf.read('/home/yangwenhao/dataset/CN-Celeb/data/id00635/live_broadcast-01-221.flac', start=0, stop=208000)

In [12]:
df = pd.read_csv(root_path + '/Data/checkpoint/ThinResNet18/Mean_batch256_k7_seblock_downk1_avg1_SAP2_em256_dp05_alpha0_none1_wde5_2sesmix8/arcsoft_adam_cyclic/cnceleb/wave_fb80_dist2/train.csv')

In [ ]:
errors = []
for i in tqdm(df.iterrows()):
    # print(i[1].uid, i[1].start, i[1].end)
    try:
        data,sr = sf.read(utt2path[i[1].uid], start=i[1].start, stop=i[1].end)
    except Exception as e:
        errors.add()